In [9]:
import pandas as pd
import numpy as np
from pprint import pprint

# Step 1: Data Preprocessing
# Load dataset
data = pd.read_csv('MushroomDataset/secondary_data.csv', delimiter=';')

# Drop columns with large missing values
filter_out = ['stem-root', 'veil-type', 'veil-color', 'spore-print-color']
filtered_data = data.drop(columns=filter_out)
fd = filtered_data.copy()

# Fill missing values for categorical features with mode
categorical_features = ['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']
for feature in categorical_features:
    fd[feature] = fd[feature].fillna(fd[feature].mode()[0])

# Fill missing values for continuous features with mean
continuous_features = ['cap-diameter', 'stem-height', 'stem-width']
for feature in continuous_features:
    fd[feature] = fd[feature].fillna(fd[feature].mean())

# Step 2: Feature Selection and Binning
# Select features
selected_features = ['cap-diameter', 'cap-shape', 'cap-surface', 'stem-height', 'stem-width']
fd = fd[selected_features + ['class']]

# Binning continuous features
fd['cap-diameter_binned'] = pd.cut(fd['cap-diameter'], bins=3, labels=['Low', 'Medium', 'High'])
fd['stem-height_binned'] = pd.cut(fd['stem-height'], bins=3, labels=['Low', 'Medium', 'High'])
fd['stem-width_binned'] = pd.cut(fd['stem-width'], bins=3, labels=['Low', 'Medium', 'High'])

# Convert categorical features to numerical codes for simplicity
for column in fd.select_dtypes(['category', 'object']):
    fd[column] = fd[column].astype('category').cat.codes

# Split data into features and target
X = fd.drop(columns=['class']).values
y = fd['class'].values

# Step 3: Data Splitting with Stratification from Scratch
def stratified_split(X, y, test_size=0.33, random_state=None):
    np.random.seed(random_state)
    unique_classes, y_indices = np.unique(y, return_inverse=True)
    train_indices = []
    test_indices = []

    for cls in unique_classes:
        cls_indices = np.where(y == cls)[0]
        np.random.shuffle(cls_indices)
        n_test = int(test_size * len(cls_indices))
        test_indices.extend(cls_indices[:n_test])
        train_indices.extend(cls_indices[n_test:])

    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

# Perform stratified split
X_train, X_test, y_train, y_test = stratified_split(X, y, test_size=0.33, random_state=42)

# Step 4: Implement Decision Tree from Scratch
def entropy(y):
    unique_classes, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    return -np.sum(probabilities * np.log2(probabilities))

def information_gain(y, y_left, y_right):
    left_weight = len(y_left) / len(y)
    right_weight = len(y_right) / len(y)
    return entropy(y) - (left_weight * entropy(y_left) + right_weight * entropy(y_right))

class DecisionTree:
    def __init__(self):
        self.tree = {}

    def _build_tree(self, X, y, depth=0):
        if len(np.unique(y)) == 1 or len(y) < self.min_samples_split or (self.max_depth and depth >= self.max_depth):
            return np.unique(y)[0]

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y):
        # Base case
        if len(np.unique(y)) == 1:
            return np.unique(y)[0]

        # Recursive case
        best_feature, best_threshold = self._best_split(X, y)
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold

        left_subtree = self._build_tree(X[left_indices], y[left_indices])
        right_subtree = self._build_tree(X[right_indices], y[right_indices])

        return {'feature': best_feature, 'threshold': best_threshold, 'left': left_subtree, 'right': right_subtree}

    def _best_split(self, X, y):
        best_gain = -1
        best_feature = None
        best_threshold = None
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices = X[:, feature] <= threshold
                right_indices = X[:, feature] > threshold
                gain = information_gain(y, y[left_indices], y[right_indices])
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold

def classify(tree, sample):
    if isinstance(tree, dict):
        feature, threshold, left, right = tree['feature'], tree['threshold'], tree['left'], tree['right']
        if sample[feature] <= threshold:
            return classify(left, sample)
        else:
            return classify(right, sample)
    else:
        return tree

# Train the model
tree = DecisionTree()
tree.fit(X_train, y_train)

# Make predictions and calculate accuracy
y_pred = [classify(tree.tree, sample) for sample in X_test]
accuracy = np.mean(y_pred == y_test)
print("Model accuracy:", accuracy)

pprint(tree.tree)

Model accuracy: 0.9322151647479159
{'feature': 4,
 'left': {'feature': 3,
          'left': {'feature': 4,
                   'left': {'feature': 4,
                            'left': 1,
                            'right': {'feature': 3,
                                      'left': 0,
                                      'right': {'feature': 4,
                                                'left': {'feature': 0,
                                                         'left': 1,
                                                         'right': 0,
                                                         'threshold': 0.82},
                                                'right': 1,
                                                'threshold': 0.75},
                                      'threshold': 2.31},
                            'threshold': 0.74},
                   'right': {'feature': 0,
                             'left': {'feature': 2,
                                   